# eEDB-011-2024-3

## Atividade 02: Ingestão e ETL com linguagem de programação (Python)

- Utilizar linguagem de programação Python para ingestão e tratamento de dados.
    - Pacotes adicionais podem ser utilizados (exceto Spark)
    - Tratamento de dados não deve ser realizado via SQL
- Realizar a ingestão de todas as base de dados em um banco de dados relacional open source. Pode ser utilizado qualquer banco de dado sendo algumas sugestões:
    - MySQL
    - Postgre
    - ClickHouse
- Gerar uma tabela final com os dados tratados e unidos.
    - O tratamento de dados deve ser realizado através da linguagem de programação Python
- Adicionar as seguintes camadas de processamento, dentro do próprio banco de dados ou em disco local. A Camada Delivery deve obrigatoriamente ter estar também no formato de uma tabela final dentro do banco de dados relacional:
    - RAW – formato dos dados livre
    - Trusted – formato de dados em Parquet ou ORC or AVRO (indicado Parquet)
    - Delivery– formato de dados em Parquet ou ORC or AVRO (indicado Parquet)

- **Grupo 02**:
    - Aline Bini
    - Ana Lívia Franco
    - Ana Priss
    - João Squinelato
    - Marcelo Pena
    - Thais Siqueira

- [Github](https://github.com/Squinelato/eEDB-011-2024-3 "eEDB-011-2024-3")

```Ingestão De Dados | Agosto 2024```

## To Do

- raw 
    - salvar em parquet (ok)
    - salvar rwzd_{tabela} (ok)
    - setar schema de str (ok)
    - criar as pastas pros parquets (ok)
- trusted
    - ler da raw (ok)
    - criar uma chave artificial em todas as tabelas trusted a partir do cálculo de hash da coluna que contém o nome da instituição financeira, porém, removendo os espaços em branco e deixando todas as letras em caixa baixa e removendo a acentuação (ok)
    - aplicar data quality (ok)
        - trocar os nomes das colunas para ingles (pegar do arquivo de DDL) (ok)
        - trocar os nomes das colunas para snakecase (ok)
        - mudar tipos (ok)
        - outras transformacoes
            - remover coluna unnamed (ok)
- delivery
    - tabela consolidada
    - agregação

- Adicionar zeros à esquerda para os CNPJs, exemplo, CNPJ do banco do é 00000000... mas no arquivo orignal está como apenas "0" (ok)
- Considerar o CPPJ com o segmento para realizar o join (ok)

---
## Library Imports

In [5]:
from unidecode import unidecode
from hashlib import sha1
import pandas as pd
import numpy as np
import os

---
## **Raw**

### **Banks file**

In [2]:
banks_csv_path = './trzd/Bancos/EnquadramentoInicia_v2_alterado_manualmente_revisado.csv'
rwzd_banks = pd.read_csv(banks_csv_path, sep='\t', encoding='utf8', dtype=str)
rwzd_banks.head()

,Segmento,CNPJ,Nome
0,S1,0,BANCO DO BRASIL - PRUDENCIAL
1,S1,60746948,BRADESCO - PRUDENCIAL
2,S1,30306294,BTG PACTUAL - PRUDENCIAL
3,S1,360305,CAIXA ECONOMICA FEDERAL - PRUDENCIAL
4,S1,60872504,ITAU - PRUDENCIAL


In [3]:
rwzd_banks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1474 entries, 0 to 1473
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Segmento  1474 non-null   object
 1   CNPJ      1474 non-null   object
 2   Nome      1474 non-null   object
dtypes: object(3)
memory usage: 34.7+ KB


In [4]:
rwzd_banks.to_parquet('./raw/bank/data.parquet', engine='pyarrow', compression='snappy')

### **Employees file**

In [6]:
employees_csv_path = './trzd/Empregados/glassdoor_consolidado_join_match_v2.csv'
rwzd_employees = pd.read_csv(employees_csv_path, sep='|', encoding='utf-8', dtype=str)
rwzd_employees.head(2)

,employer_name,reviews_count,culture_count,salaries_count,benefits_count,employer-website,employer-headquarters,employer-founded,employer-industry,employer-revenue,...,Diversidade e inclusão,Qualidade de vida,Alta liderança,Remuneração e benefícios,Oportunidades de carreira,Recomendam para outras pessoas(%),Perspectiva positiva da empresa(%),Segmento,Nome,match_percent
0,BNP Paribas,13000,4100,20000,3600,http://www.group.bnpparibas,"Paris, França",2000.0,/Explorar/melhores-serviços-bancários-e-de-cré...,Mais de US$ 10 bilhões,...,4.0,3.8,3.4,3.4,3.5,77.0,63.0,S3,BNP PARIBAS,100
1,BTG Pactual,1600,683,2800,635,http://www.btgpactual.com,"São Paulo, Brasil",NaN,/Explorar/melhores-gestão-de-ativos-e-investim...,Desconhecida/não se aplica,...,3.5,2.8,3.7,4.4,4.2,78.0,73.0,S1,BTG PACTUAL,100


In [7]:
rwzd_employees.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 23 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   employer_name                       34 non-null     object
 1   reviews_count                       34 non-null     object
 2   culture_count                       34 non-null     object
 3   salaries_count                      34 non-null     object
 4   benefits_count                      34 non-null     object
 5   employer-website                    34 non-null     object
 6   employer-headquarters               34 non-null     object
 7   employer-founded                    28 non-null     object
 8   employer-industry                   34 non-null     object
 9   employer-revenue                    34 non-null     object
 10  url                                 34 non-null     object
 11  Geral                               34 non-null     object
 

In [8]:
rwzd_employees.to_parquet('./raw/employee/data.parquet', engine='pyarrow', compression='snappy')

### **Claims**

Carregando todos os arquivos de reclamações e os unindo em uma única tabela

In [9]:
list_df_claims = []
claims_dir = './trzd/Reclamações'

for file in os.listdir(claims_dir):
    full_path = os.path.join(claims_dir, file)
    print(f'Processing file: {full_path}')

    claim = pd.read_csv(full_path, sep=';', encoding='utf-8', dtype=str)
    list_df_claims.append(claim)

rwzd_claims = pd.concat(list_df_claims)

Processing file: ./trzd/Reclamações\2021_tri_01.csv
Processing file: ./trzd/Reclamações\2021_tri_02.csv
Processing file: ./trzd/Reclamações\2021_tri_03.csv
Processing file: ./trzd/Reclamações\2021_tri_04.csv
Processing file: ./trzd/Reclamações\2022_tri_01.csv
Processing file: ./trzd/Reclamações\2022_tri_03.csv
Processing file: ./trzd/Reclamações\2022_tri_04.csv


In [10]:
rwzd_claims.head(2)

,Ano,Trimestre,Categoria,Tipo,CNPJ IF,Instituição financeira,Índice,Quantidade de reclamações reguladas procedentes,Quantidade de reclamações reguladas - outras,Quantidade de reclamações não reguladas,Quantidade total de reclamações,Quantidade total de clientes – CCS e SCR,Quantidade de clientes – CCS,Quantidade de clientes – SCR,Unnamed: 14
0,2021,1º,Grupo Secundário,Conglomerado,,ABC-BRASIL (conglomerado),,2,3,4,9,26230,24698,3810,NaN
1,2021,1º,Grupo Secundário,Conglomerado,,AGIBANK (conglomerado),"54,79",58,140,73,271,1058431,790848,693843,NaN


In [11]:
rwzd_claims.info()

<class 'pandas.core.frame.DataFrame'>
Index: 918 entries, 0 to 153
Data columns (total 15 columns):
 #   Column                                           Non-Null Count  Dtype 
---  ------                                           --------------  ----- 
 0   Ano                                              918 non-null    object
 1   Trimestre                                        918 non-null    object
 2   Categoria                                        918 non-null    object
 3   Tipo                                             918 non-null    object
 4   CNPJ IF                                          918 non-null    object
 5   Instituição financeira                           918 non-null    object
 6   Índice                                           918 non-null    object
 7   Quantidade de reclamações reguladas procedentes  918 non-null    object
 8   Quantidade de reclamações reguladas - outras     918 non-null    object
 9   Quantidade de reclamações não reguladas         

In [12]:
rwzd_claims.to_parquet('./raw/claim/data.parquet', engine='pyarrow', compression='snappy')

---
## **Trusted**

### **Banks**

In [13]:
rwzd_bank_path = './raw/bank'
trzd_bank = pd.read_parquet(rwzd_bank_path, engine='pyarrow')

Aplicando algumas transformações com o intuito de melhorar a qualidade dos dados:

1 - Renomeando colunas do _dataframe_ para inglês e no formato _snake case_

In [14]:
trzd_bank.rename(
    columns={
        'Segmento': 'segment',
        'CNPJ': 'cnpj',
        'Nome': 'financial_institution_name'
    },
    inplace = True
)

2 - Para que os dados da coluna _cnpj_ estivessem de acordo com seu padrão, os valores incompletos receberam numerais zeros à esquerda até completar 8 dígitos

In [15]:
trzd_bank["cnpj"] = trzd_bank["cnpj"].map(lambda cnpj: cnpj.zfill(8))

3 - Criando chave artificial com base no CNPJ e o Segmento da instituição financeira

In [16]:
trzd_bank["bank_document_id"] = trzd_bank["cnpj"] + trzd_bank["segment"]
trzd_bank["bank_document_id"] = trzd_bank["bank_document_id"].map(lambda id: sha1(id.encode("utf-8")).hexdigest())

In [17]:
trzd_bank.head()

,segment,cnpj,financial_institution_name,bank_document_id
0,S1,00000000,BANCO DO BRASIL - PRUDENCIAL,d9be4941c63af670d63086dcb0849a997e062a64
1,S1,60746948,BRADESCO - PRUDENCIAL,2f59bebc86f6b4dd5628043cac240c6e6a6f651d
2,S1,30306294,BTG PACTUAL - PRUDENCIAL,110ffbf347c0837827daf658f4f97950ebe9f741
3,S1,00360305,CAIXA ECONOMICA FEDERAL - PRUDENCIAL,587e4dca521e8c4426371a29658419ecb0087262
4,S1,60872504,ITAU - PRUDENCIAL,4b6e5b20d113f3415704fd6a998f2e4500340b18


Armazendo o dataframe na camada Trusted

In [20]:
trzd_bank.to_parquet('./trusted/bank/data.parquet', engine='pyarrow', compression='snappy')

### **Employees**

In [21]:
rwzd_employee_path = './raw/employee'
trzd_employee = pd.read_parquet(rwzd_employee_path, engine='pyarrow')

Aplicando algumas transformações com o intuito de melhorar a qualidade dos dados:

1 - Renomeando colunas do _dataframe_ para inglês e no formato _snake case_

In [22]:
trzd_employee.rename(
    columns={
        'employer-website': 'employer_website',
        'employer-headquarters': 'employer_headquarters',
        'employer-founded': 'employer_founded',
        'employer-industry': 'employer_industry',
        'employer-revenue': 'employer_revenue',
        'Geral': 'general_score',
        'Cultura e valores': 'culture_values_score',
        'Diversidade e inclusão': 'diversity_inclusion_score',
        'Qualidade de vida': 'life_quality_score',
        'Alta liderança': 'senior_leadership_score',
        'Remuneração e benefícios': 'compensation_benefits_score',
        'Oportunidades de carreira': 'career_opportunities_score',
        'Recomendam para outras pessoas(%)': 'recommendation_score',
        'Perspectiva positiva da empresa(%)': 'company_positive_score',
        'Segmento': 'segment',
        'Nome': 'financial_institution_name'
    },
    inplace=True
)

2 - Alterando os tipos das colunas do _dataframe_ para tipos mais _apropriados_

In [23]:
trzd_employee = trzd_employee.astype({
    'employer_name': 'str',
    'reviews_count': 'uint64',
    'culture_count': 'uint64',
    'salaries_count': 'uint64',
    'benefits_count': 'uint64',
    'employer_website': 'str',
    'employer_headquarters': 'str',
    'employer_founded': 'str',
    'employer_industry': 'str',
    'employer_revenue': 'str',
    'url': 'str',
    'general_score': 'float64',
    'culture_values_score': 'float64',
    'diversity_inclusion_score': 'float64',
    'life_quality_score': 'float64',
    'senior_leadership_score': 'float64',
    'compensation_benefits_score': 'float64',
    'career_opportunities_score': 'float64',
    'recommendation_score': 'float64',
    'company_positive_score': 'float64',
    'segment': 'str',
    'financial_institution_name': 'str',
    'match_percent': 'uint8'
})

3 - Tratando campo de data de fundação para remover casas decimais

In [24]:
trzd_employee["employer_founded"] =  trzd_employee["employer_founded"].str.replace('\..*', '', regex=True)

4 - Removendo acentos do campos _financial_institution_name_

In [25]:
trzd_employee['financial_institution_name'] = trzd_employee['financial_institution_name'].map(lambda x: unidecode(x))

5 - Criando chave artificial com base no nome da instituição financeira

In [26]:
trzd_employee['bank_name_id'] = trzd_employee['financial_institution_name'].str.lower()
trzd_employee['bank_name_id'] = trzd_employee['bank_name_id'].str.replace(" ", "")
trzd_employee['bank_name_id'] = trzd_employee['bank_name_id'].map(lambda x: sha1(x.encode('utf-8')).hexdigest())

In [27]:
trzd_employee.head()

,employer_name,reviews_count,culture_count,salaries_count,benefits_count,employer_website,employer_headquarters,employer_founded,employer_industry,employer_revenue,...,life_quality_score,senior_leadership_score,compensation_benefits_score,career_opportunities_score,recommendation_score,company_positive_score,segment,financial_institution_name,match_percent,bank_name_id
0,BNP Paribas,13000,4100,20000,3600,http://www.group.bnpparibas,"Paris, França",2000,/Explorar/melhores-serviços-bancários-e-de-cré...,Mais de US$ 10 bilhões,...,3.8,3.4,3.4,3.5,77.0,63.0,S3,BNP PARIBAS,100,0612d12173005ce00d9c10800a27985a1e75997e
1,BTG Pactual,1600,683,2800,635,http://www.btgpactual.com,"São Paulo, Brasil",None,/Explorar/melhores-gestão-de-ativos-e-investim...,Desconhecida/não se aplica,...,2.8,3.7,4.4,4.2,78.0,73.0,S1,BTG PACTUAL,100,cdd80a41f223ad9a4236b42c6c1c465c41908aed
2,Banco Alfa,175,74,271,105,http://www.alfanet.com.br,"São Paulo, Brasil",1925,/Explorar/melhores-serviços-bancários-e-de-cré...,Desconhecida/não se aplica,...,3.1,2.8,3.8,2.7,68.0,47.0,S3,ALFA,100,1f7d72cc0ecb87cb6225c2979f3ccbeaf7cd0c33
3,Banco BMG,445,232,704,277,http://www.bancobmg.com.br,"São Paulo, Brasil",1930,/Explorar/melhores-serviços-bancários-e-de-cré...,De US$ 1 a US$ 5 milhões,...,3.7,3.6,4.2,3.5,79.0,63.0,S3,BMG,100,c89e80ee154b6215c5dc790d93590f620e1ec45d
4,Banco Bradesco,11000,3300,19000,3200,http://www.bancobradesco.com.br,"Osasco, Brasil",1943,/Explorar/melhores-serviços-bancários-e-de-cré...,Mais de US$ 10 bilhões,...,3.4,3.4,4.3,3.8,81.0,66.0,S1,BRADESCO,100,dd2b7d97678a3cb8b5a9f5e5bf60907d29293298


Armazendo o dataframe na camada Trusted

In [28]:
trzd_employee.to_parquet('./trusted/employee/data.parquet', engine='pyarrow', compression='snappy')

### **Claims**

In [29]:
rwzd_claim_path = './raw/claim'
trzd_claim = pd.read_parquet(rwzd_claim_path, engine='pyarrow')

Aplicando algumas transformações com o intuito de melhorar a qualidade dos dados:

1 - Removendo coluna sem dados

In [30]:
trzd_claim.drop(columns=['Unnamed: 14'], inplace=True)

2 - Renomendo colunas do _dataframe_ para inglês e no formato _snake case_

In [31]:
trzd_claim.rename(
    columns={
        'Ano': 'year',
        'Trimestre': 'quarter',
        'Categoria': 'category',
        'Tipo': 'type',
        'CNPJ IF': 'cnpj',
        'Instituição financeira': 'financial_institution_name',
        'Índice': 'index',
        'Quantidade de reclamações reguladas procedentes': 'number_of_regulated_proceeding_complaints',
        'Quantidade de reclamações reguladas - outras': 'number_of_regulated_other_complaints',
        'Quantidade de reclamações não reguladas': 'number_of_unregulated_complaints',
        'Quantidade total de reclamações': 'total_number_of_complaints',
        'Quantidade total de clientes – CCS e SCR': 'total_number_of_ccs_and_scr_customers',
        'Quantidade de clientes – CCS': 'number_of_ccs_customers',
        'Quantidade de clientes – SCR': 'number_of_scr_customers'
    },
    inplace=True
)

3 - Tratando a coluna _index_

- Devido a valores com separação por pontos na cada do milhar, foi necessário realizar a remoção (e.g.: 2.000,00 passa a ser 2000,00)
- A fim de converter mais tarde para o tipo float, a vírgula da casa dos decimais foi substituída por ponto
- Por fim, string vazias foram substituídas por NaN

In [32]:
trzd_claim["index"] = trzd_claim["index"].str.replace('.', '')
trzd_claim["index"] = trzd_claim["index"].str.replace(',', '.')
trzd_claim["index"] = trzd_claim["index"].replace(' ', np.nan)

4 - Tratando as colunas _quarter_ e _cnpj_

- O caracter º foi removido da coluna _quarter_
- Para que os dados da coluna _cnpj_ estivessem de acordo com seu padrão, os valores incompletos receberam numerais zeros à esquerda até completar 8 dígitos

In [33]:
trzd_claim["quarter"] = trzd_claim["quarter"].str.replace('º', '')
trzd_claim["cnpj"] = trzd_claim["cnpj"].map(lambda cnpj: cnpj.zfill(8))

5 - Tratando as colunas _total_number_of_ccs_and_scr_customers_, _number_of_ccs_customers_ e _number_of_scr_customers_, as quais continham strings vazias

Assim, as strings vazias foram substituídas por valores NaN e convertidas para float, possibilitando a conversão para o tipo inteiro mais adiante

In [34]:
trzd_claim['total_number_of_ccs_and_scr_customers'] = trzd_claim['total_number_of_ccs_and_scr_customers'].replace(' ', np.nan).astype(float)
trzd_claim['number_of_ccs_customers'] = trzd_claim['number_of_ccs_customers'].replace(' ', np.nan).astype(float)
trzd_claim['number_of_scr_customers'] = trzd_claim['number_of_scr_customers'].replace(' ', np.nan).astype(float)

6 - Alterando os tipos das colunas do _dataframe_ para tipos mais _apropriados_

Devido ao tipo uint64 não aceitar valores NaN, as colunas que continham estes valores precisaram ser convertidas ao tipo inteiro do Pandas, que aceita

In [35]:
trzd_claim = trzd_claim.astype({
    'year': 'str',
    'quarter': 'uint8',
    'category': 'str',
    'type': 'str',
    'cnpj': 'str',
    'financial_institution_name': 'str',
    'index': 'float64',
    'number_of_regulated_proceeding_complaints': 'uint64',
    'number_of_regulated_other_complaints': 'uint64',
    'number_of_unregulated_complaints': 'uint64',
    'total_number_of_complaints': 'uint64',
    'total_number_of_ccs_and_scr_customers': pd.Int64Dtype(),
    'number_of_ccs_customers': pd.Int64Dtype(),
    'number_of_scr_customers': pd.Int64Dtype()
})

7 - Padronizando o campo _financial_institution_name_

Padrão encontrado " (conglomerado)", que não estava de acordo com o campo _financial_institution_name_ em _employees_

In [36]:
trzd_claim['financial_institution_name'] = trzd_claim['financial_institution_name'].str.replace(' (conglomerado)', '')

Remoção de acentos para aumentar a probabilidade de match com os bancos de _ employees_

In [37]:
trzd_claim['financial_institution_name'] = trzd_claim['financial_institution_name'].map(lambda x: unidecode(x))

Casos pontuais encontrados nos nomes

In [38]:
trzd_claim['financial_institution_name'] = trzd_claim['financial_institution_name'].str.replace('BB', 'BANCO DO BRASIL')
trzd_claim['financial_institution_name'] = trzd_claim['financial_institution_name'].str.replace('DAYCOVAL', 'BANCO DAYCOVAL S.A')
trzd_claim['financial_institution_name'] = trzd_claim['financial_institution_name'].str.replace('DEUTSCHE BANK S.A. - BANCO ALEMAO', 'DEUTSCHE')
trzd_claim['financial_institution_name'] = trzd_claim['financial_institution_name'].str.replace('BANCO SUMITOMO MITSUI BRASILEIRO S.A.', 'BANCO SUMITOMO MITSUI BRASIL S.A.')

8 - Criando duas chaves artificiais: uma com base no nome da instituição financeira e outra com base no CNPJ e o Segmento da instituição financeira

In [39]:
trzd_claim['bank_name_id'] = trzd_claim['financial_institution_name'].str.lower()
trzd_claim['bank_name_id'] = trzd_claim['bank_name_id'].str.replace(" ", "")
trzd_claim['bank_name_id'] = trzd_claim['bank_name_id'].map(lambda x: sha1(x.encode('utf-8')).hexdigest())

In [40]:
trzd_claim.head()

,year,quarter,category,type,cnpj,financial_institution_name,index,number_of_regulated_proceeding_complaints,number_of_regulated_other_complaints,number_of_unregulated_complaints,total_number_of_complaints,total_number_of_ccs_and_scr_customers,number_of_ccs_customers,number_of_scr_customers,bank_name_id
0,2021,1,Grupo Secundário,Conglomerado,0000000,ABC-BRASIL,NaN,2,3,4,9,26230,24698,3810,e59e75522e2fc29a497ee94dfac733ebe7557b68
1,2021,1,Grupo Secundário,Conglomerado,0000000,AGIBANK,54.79,58,140,73,271,1058431,790848,693843,9935e071203e03fbcf8bebc6bf6eed7f348e3a7a
2,2021,1,Grupo Secundário,Banco/financeira,36321990,"AGORACRED S/A SOCIEDADE DE CREDITO, FINANCIAME...",NaN,3,3,0,6,420692,129,420563,d14cba42359f43b5fbd61ee3d0faefaa73997b85
3,2021,1,Grupo Secundário,Banco/financeira,27214112,"AL5 S.A. CREDITO, FINANCIAMENTO E INVESTIMENTO",NaN,1,1,0,2,12645,4979,10112,c01ccc940cb107c1e48d2eef610c91c7d0354b2e
4,2021,1,Grupo Secundário,Conglomerado,0000000,ALFA,NaN,14,44,15,73,412135,268186,145105,1f7d72cc0ecb87cb6225c2979f3ccbeaf7cd0c33


Armazendo o dataframe na camada Trusted

In [41]:
trzd_claim.to_parquet('./trusted/claim/data.parquet', engine='pyarrow', compression='snappy')

---
## Delivery

- selecionar o que queremos para não repetir colunas (no próprio merge decide)
- salvar em parquet + to_sql (colocar apenas a tabela consolidada no banco)
- sugestão: instalar postgree para criar o banco

In [141]:
trzd_bank_path = './trusted/bank'
banks = pd.read_parquet(trzd_bank_path, engine='pyarrow')

trzd_employee_path = './trusted/employee'
employee = pd.read_parquet(trzd_employee_path, engine='pyarrow')

trzd_claim_path = './trusted/claim'
claims = pd.read_parquet(trzd_claim_path, engine='pyarrow')

**Junção entre as tabelas banks e claim através do cnpj**

A coluna _financial_institution_name_ foi removida de banks para que não houvesse repetição de coluna

Como esta coluna em claims foi ajustado para estar compatível com employees, foi a escolhida para manter-se no merge

In [145]:
banks = banks.drop(columns=['financial_institution_name'])

banks_claims = pd.merge(claims, banks, on='cnpj', how='inner')

banks_claims.head(2)

,year,quarter,category,type,cnpj,financial_institution_name,index,number_of_regulated_proceeding_complaints,number_of_regulated_other_complaints,number_of_unregulated_complaints,total_number_of_complaints,total_number_of_ccs_and_scr_customers,number_of_ccs_customers,number_of_scr_customers,bank_name_id,segment,bank_document_id
0,2021,1,Grupo Secundário,Banco/financeira,36321990,"AGORACRED S/A SOCIEDADE DE CREDITO, FINANCIAME...",NaN,3,3,0,6,420692,129,420563,d14cba42359f43b5fbd61ee3d0faefaa73997b85,S4,b5a8f0808f253ba305496155fa6fd50f449e5846
1,2021,1,Grupo Secundário,Banco/financeira,03532415,BANCO ABN AMRO S.A.,NaN,0,0,3,3,98,71,31,c444a43000c079d0bf4eb83a1dcedefc66e180e7,S4,a6966623c9b8e7a3352769e1118e56da6dd3ecb5


In [146]:
banks_claims.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 327 entries, 0 to 326
Data columns (total 17 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   year                                       327 non-null    object 
 1   quarter                                    327 non-null    uint8  
 2   category                                   327 non-null    object 
 3   type                                       327 non-null    object 
 4   cnpj                                       327 non-null    object 
 5   financial_institution_name                 327 non-null    object 
 6   index                                      47 non-null     float64
 7   number_of_regulated_proceeding_complaints  327 non-null    uint64 
 8   number_of_regulated_other_complaints       327 non-null    uint64 
 9   number_of_unregulated_complaints           327 non-null    uint64 
 10  total_number_of_complaints

**Junção entre as tabelas claim e employees através do bank_name_id**

A coluna _financial_institution_name_ foi removida de employee para que não houvesse repetição de coluna

Como esta coluna em claims possui mais linhas, foi a escolhida para manter-se no merge

In [147]:
employee = employee.drop(columns=['financial_institution_name'])

employee_claims = pd.merge(claims, employee, on='bank_name_id', how='inner')

employee_claims.head(2)

,year,quarter,category,type,cnpj,financial_institution_name,index,number_of_regulated_proceeding_complaints,number_of_regulated_other_complaints,number_of_unregulated_complaints,total_number_of_complaints,total_number_of_ccs_and_scr_customers,number_of_ccs_customers,number_of_scr_customers,bank_name_id,employer_name,reviews_count,culture_count,salaries_count,benefits_count,employer_website,employer_headquarters,employer_founded,employer_industry,employer_revenue,url,general_score,culture_values_score,diversity_inclusion_score,life_quality_score,senior_leadership_score,compensation_benefits_score,career_opportunities_score,recommendation_score,company_positive_score,segment,match_percent
0,2021,1,Grupo Secundário,Conglomerado,0000000,ALFA,NaN,14,44,15,73,412135,268186,145105,1f7d72cc0ecb87cb6225c2979f3ccbeaf7cd0c33,Banco Alfa,175,74,271,105,http://www.alfanet.com.br,"São Paulo, Brasil",1925,/Explorar/melhores-serviços-bancários-e-de-cré...,Desconhecida/não se aplica,https://www.glassdoor.com.br/Vis%C3%A3o-geral/...,3.5,3.2,3.2,3.1,2.8,3.8,2.7,68.0,47.0,S3,100
1,2021,1,Grupo Secundário,Banco/financeira,07237373,BANCO DO NORDESTE DO BRASIL S.A.,NaN,19,8,20,47,8512373,7928582,3161435,bb953b4df7711bcfb088e1ded4bf13761c4c5aed,Banco do Nordeste do Brasil,525,113,840,115,http://www.bnb.gov.br,"Fortaleza, Brasil",None,/Explorar/melhores-gestão-de-ativos-e-investim...,Desconhecido/Não se aplica,https://www.glassdoor.com.br/Vis%C3%A3o-geral/...,4.5,4.4,4.3,4.3,4.1,4.2,4.0,90.0,83.0,S2,100


Criando chave artificial com base no CNPJ e o Segmento da instituição financeira

In [149]:
employee_claims["bank_document_id"] = employee_claims["cnpj"] + employee_claims["segment"]
employee_claims["bank_document_id"] = employee_claims["bank_document_id"].map(lambda id: sha1(id.encode("utf-8")).hexdigest())

In [150]:
employee_claims.head(2)

,year,quarter,category,type,cnpj,financial_institution_name,index,number_of_regulated_proceeding_complaints,number_of_regulated_other_complaints,number_of_unregulated_complaints,total_number_of_complaints,total_number_of_ccs_and_scr_customers,number_of_ccs_customers,number_of_scr_customers,bank_name_id,employer_name,reviews_count,culture_count,salaries_count,benefits_count,employer_website,employer_headquarters,employer_founded,employer_industry,employer_revenue,url,general_score,culture_values_score,diversity_inclusion_score,life_quality_score,senior_leadership_score,compensation_benefits_score,career_opportunities_score,recommendation_score,company_positive_score,segment,match_percent,bank_document_id
0,2021,1,Grupo Secundário,Conglomerado,0000000,ALFA,NaN,14,44,15,73,412135,268186,145105,1f7d72cc0ecb87cb6225c2979f3ccbeaf7cd0c33,Banco Alfa,175,74,271,105,http://www.alfanet.com.br,"São Paulo, Brasil",1925,/Explorar/melhores-serviços-bancários-e-de-cré...,Desconhecida/não se aplica,https://www.glassdoor.com.br/Vis%C3%A3o-geral/...,3.5,3.2,3.2,3.1,2.8,3.8,2.7,68.0,47.0,S3,100,c40a791a9bd30fc7c09cb28deafc17739e2164f3
1,2021,1,Grupo Secundário,Banco/financeira,07237373,BANCO DO NORDESTE DO BRASIL S.A.,NaN,19,8,20,47,8512373,7928582,3161435,bb953b4df7711bcfb088e1ded4bf13761c4c5aed,Banco do Nordeste do Brasil,525,113,840,115,http://www.bnb.gov.br,"Fortaleza, Brasil",None,/Explorar/melhores-gestão-de-ativos-e-investim...,Desconhecido/Não se aplica,https://www.glassdoor.com.br/Vis%C3%A3o-geral/...,4.5,4.4,4.3,4.3,4.1,4.2,4.0,90.0,83.0,S2,100,4e52ea413cf937405f38013294434e85be954f2d


In [151]:
employee_claims.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154 entries, 0 to 153
Data columns (total 38 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   year                                       154 non-null    object 
 1   quarter                                    154 non-null    uint8  
 2   category                                   154 non-null    object 
 3   type                                       154 non-null    object 
 4   cnpj                                       154 non-null    object 
 5   financial_institution_name                 154 non-null    object 
 6   index                                      85 non-null     float64
 7   number_of_regulated_proceeding_complaints  154 non-null    uint64 
 8   number_of_regulated_other_complaints       154 non-null    uint64 
 9   number_of_unregulated_complaints           154 non-null    uint64 
 10  total_number_of_complaints

**Junção entre as tabelas banks_claims e employee_claims através do bank_document_id**

Verificando quais as colunas que employee_claims não possui em comum com banks_claims para evitar repetição de colunas ao realizar a junção

As colunas de banks_claims foram mantidas pois é a que possui maior quantidade de linhas

In [152]:
set(employee_claims.columns) - set(banks_claims.columns)

{'benefits_count',
 'career_opportunities_score',
 'company_positive_score',
 'compensation_benefits_score',
 'culture_count',
 'culture_values_score',
 'diversity_inclusion_score',
 'employer_founded',
 'employer_headquarters',
 'employer_industry',
 'employer_name',
 'employer_revenue',
 'employer_website',
 'general_score',
 'life_quality_score',
 'match_percent',
 'recommendation_score',
 'reviews_count',
 'salaries_count',
 'senior_leadership_score',
 'url'}

Junção das tabelas

In [153]:
banks_consolidated = pd.merge(
    banks_claims, 
    employee_claims[['bank_document_id', 'benefits_count', 'career_opportunities_score', 'company_positive_score', 'compensation_benefits_score', 'culture_count', 'culture_values_score', 'diversity_inclusion_score', 'employer_founded', 'employer_headquarters', 'employer_industry', 'employer_name', 'employer_revenue', 'employer_website', 'general_score', 'life_quality_score', 'match_percent', 'recommendation_score', 'reviews_count', 'salaries_count', 'senior_leadership_score', 'url']], 
    on='bank_document_id', 
    how='inner'
)

In [155]:
banks_consolidated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 38 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   year                                       126 non-null    object 
 1   quarter                                    126 non-null    uint8  
 2   category                                   126 non-null    object 
 3   type                                       126 non-null    object 
 4   cnpj                                       126 non-null    object 
 5   financial_institution_name                 126 non-null    object 
 6   index                                      0 non-null      float64
 7   number_of_regulated_proceeding_complaints  126 non-null    uint64 
 8   number_of_regulated_other_complaints       126 non-null    uint64 
 9   number_of_unregulated_complaints           126 non-null    uint64 
 10  total_number_of_complaints

In [169]:
# salvei em csv só pra conseguir visualizar melhor rs depois removemos
banks_consolidated.to_csv('banks_consolidated.csv', sep=';')

Inserção no banco de dados

Doc: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html

---
## Gerar arquivo python
Exemplo

In [62]:
%%writefile ./scripts/testes.py
print('hello world')

Writing ./scripts/testes.pyg
